In [1]:
import time
import pandas as pd
import pdfplumber
import re
from collections import namedtuple


In [2]:
input_pdf_path = "../Data/PDF/Test_In/Invoice-17249-1-LATEST.pdf"
reference_csv_path = "../Data/CSV/Test_In/Invoice-17249-1-LATEST.csv"

In [3]:
ref_csv_df = pd.read_csv(reference_csv_path)

In [4]:
ref_csv_df.head()

,PDF_Header,PDF_key,Key_type
0,NaN,Invoice,table
1,NaN,Due,table
2,NaN,Customer,table
3,NaN,Invoice,table
4,top,Line,table


In [9]:
data = dict(zip(ref_csv_df.PDF_Header,ref_csv_df.PDF_key))
data

{nan: 'Invoice', 'top': 'Line', 'bottom': 'KL-Test7', 'right': 'Amount(Net)'}

In [ ]:
full_text=[]
full_words=[]
with pdfplumber.open(path) as pdf_obj:
    pages = pdf_obj.pages
    for i in pages:
        text = i.extract_text()#This gives lines from the document separated with new line '\n'
        full_text.append(text)        
        words = i.extract_words()
        full_words.append(words)

In [ ]:
full_text

In [ ]:
full_words

In [ ]:
for text in full_text:
    for line in text.split('\n'):
        print(line)

In [5]:
with pdfplumber.open(input_pdf_path) as pdf:
    first_page = pdf.pages[0]
    rows = first_page.extract_text().split('\n')
rows[:10]

['Invoice',
 'Invoice Date Due Date Customer # Invoice #',
 '11/11/21 12/11/21 1000001 15803-00',
 'PO Date PO # Page #',
 '11/11/21 1',
 'Bill To Ship To',
 'KL Cust2 KL Cust2',
 '3001 N NEVADA AVE 3001 N NEVADA AVE',
 'COLORADO SPRINGS, CO 80907 COLORADO SPRINGS, CO 80907',
 'Correspondence To']

In [6]:
first_page.to_image(resolution=150).outline_words()

ImportError: MagickWand shared library not found.
You probably had not installed ImageMagick library.
Try to install:
  https://docs.wand-py.org/en/latest/guide/install.html#install-imagemagick-on-windows

In [ ]:
data

In [10]:
with pdfplumber.open(input_pdf_path) as pdf:
    first_page = pdf.pages[0]
    rows = first_page.extract_words()

for row in rows:
    if row['text'] == data['top']:
        x0 = row['x0']
        top = row['top']
    if row['text'] == data['bottom']:
        bottom = row['bottom']        
    if row['text'] == data['right']:      
        x1 = row['x1']
       
        

box = (x0, top, x1, bottom)
box

(32.40999985, 397.63499144, 588.0489877900001, 488.12499312)

In [12]:
with pdfplumber.open(input_pdf_path) as pdf:
    first_page = pdf.pages[0]
    page = first_page.crop(bbox=(box))  # (x0, top, x1, bottom)
    table = page.extract_table(table_settings={
        "vertical_strategy": "text",
        "horizontal_strategy": "text",
    })

In [ ]:
# Let's take a look at the cropped table
page.to_image(resolution=150)

In [13]:
for row in table[:8]:
    print(row)

['Line', 'Product and Description', 'UPC', 'Order', 'Backorder', 'Shipped', 'Qty UM', 'Unit', 'Price', 'Discount', 'Amount(Net)']
['', '', 'Item', 'Quantity', 'Quantity', 'Quantity', '', 'Price', 'UM', 'Multiplier', '']
['', '', '', '', '', '', '', '', '', '', '']
['1', '1-1050', '00000', '1.00', '0.00', '1.00', 'each', '65.00', 'each', '0.00', '65.00']
['', 'KL-Test5', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '', '']
['2', '1-1051', '00000', '1.00', '0.00', '1.00', 'ea', '10.00', 'ea', '0.00', '10.00']
['', 'KL-Test6', '', '', '', '', '', '', '', '', '']


In [14]:
# Fis the blank rows by including only those rows that do not all equal blank strings
table = [row for row in table if ''.join([str(i) for i in row]) != '']

In [15]:
df = pd.DataFrame(table)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,Line,Product and Description,UPC,Order,Backorder,Shipped,Qty UM,Unit,Price,Discount,Amount(Net)
1,,,Item,Quantity,Quantity,Quantity,,Price,UM,Multiplier,
2,1,1-1050,00000,1.00,0.00,1.00,each,65.00,each,0.00,65.00
3,,KL-Test5,,,,,,,,,
4,2,1-1051,00000,1.00,0.00,1.00,ea,10.00,ea,0.00,10.00


In [17]:
Nan_header=df[df.isnull().any(axis=1)]
Nan_header

,0,1,2,3,4,5,6,7,8,9,10


In [18]:
key = Nan_header["PDF_key"].to_list()

KeyError: 'PDF_key'

In [20]:
key

['Invoice', 'Due', 'Customer', 'Invoice']

In [50]:
top=[]
with pdfplumber.open(path) as pdf:
    first_page = pdf.pages[0]
    words = first_page.extract_words()
    for word in words:
        if word['text'] in key:
            print(word['text'],word['x0'])
            top.append(word['x0'])
top

Invoice 540.8089904799999
Invoice 248.3999939
Due 336.25
Customer 424.1000061
Invoice 511.95001221


[540.8089904799999, 248.3999939, 336.25, 424.1000061, 511.95001221]

In [ ]:
for t in top:
    for row in rows:
        if row['x0']==t:
            print(row['text'],row['x0'],row['top'],row['bottom'])

In [23]:
key

['Invoice', 'Due', 'Customer', 'Invoice']

In [51]:
top = top[1:]
top

[248.3999939, 336.25, 424.1000061, 511.95001221]

In [31]:
words

[{'text': 'Invoice',
  'x0': 540.8089904799999,
  'x1': 589.05299048,
  'top': 35.447999539999955,
  'doctop': 35.447999539999955,
  'bottom': 49.447999539999955,
  'upright': True,
  'direction': 1},
 {'text': 'Invoice',
  'x0': 248.3999939,
  'x1': 272.5219939,
  'top': 117.59900358000004,
  'doctop': 117.59900358000004,
  'bottom': 124.59900358000004,
  'upright': True,
  'direction': 1},
 {'text': 'Date',
  'x0': 274.4679939,
  'x1': 289.6369939,
  'top': 117.59900358000004,
  'doctop': 117.59900358000004,
  'bottom': 124.59900358000004,
  'upright': True,
  'direction': 1},
 {'text': 'Due',
  'x0': 336.25,
  'x1': 349.473,
  'top': 117.59900358000004,
  'doctop': 117.59900358000004,
  'bottom': 124.59900358000004,
  'upright': True,
  'direction': 1},
 {'text': 'Date',
  'x0': 351.419,
  'x1': 366.588,
  'top': 117.59900358000004,
  'doctop': 117.59900358000004,
  'bottom': 124.59900358000004,
  'upright': True,
  'direction': 1},
 {'text': 'Customer',
  'x0': 424.1000061,
  'x1':

In [ ]:
key_value = namedtuple('key_value', 'Key Value')
key_value_tuple_list = []

In [60]:
def parse_column(x0, words):
    head_value_list = []
    for word in words:
        if(word['x0'] == x0):
            head_value_list.append(word['text'])
#             print(word['text'])
    
#     print(head_value_list)
    key_value = namedtuple('key_value', 'Key Value')
    key_value_tuple_list = []
    
    list_len = len(head_value_list)
    cnt_head = 0
    cnt_val = cnt_head + 1
    while cnt_val < list_len:
        nm_tuple = key_value(head_value_list[cnt_head], head_value_list[cnt_val])
        key_value_tuple_list.append(nm_tuple)
        cnt_head += 2;
        cnt_val = cnt_head + 1
        
    return key_value_tuple_list

In [61]:
top[0]

248.3999939

In [63]:
tup_list = parse_column(top[3], words)
# print(tup_list)
key_val_df = pd.DataFrame(tup_list)
key_val_df.head()

,Key,Value
0,Invoice,15803-00
1,Page,1


In [ ]:
dicty={}
listy=[]
for t in top:
    for row in rows:
        if row['x0']==t:
            dicty[row['text']+'_'+str(row['x0'])[:4]]=0
            if not listy:
                listy.append(row['bottom']+1)
            else:
                if row['top']<listy[0]
            
        
dicty
        

In [ ]:
top=top[1:]
top

In [ ]:
with pdfplumber.open(path)as pf:
    line = pf.pages[0].extract_words(x_tolerance=5)

In [ ]:
new_string=[]
new_num=[]
for i in line:
    if i['x0'] in top:
        if i['text'].isalpha():
            new_string.append(i['text'])
        else:
            new_num.append(i['text'])
new_string

In [ ]:
# new = []
# for i in new_string:
#     if i not in new:
#         new.append(i)
# new_string=new

In [ ]:
new_num

In [ ]:
data=list(zip(new_string,new_num))
data

In [ ]:
full_string={}
with pdfplumber.open(path) as pdf:
    first_page = pdf.pages[0]
    rows = first_page.extract_words()
    for row in rows:
        for data in top:
            if data == row['x0']:
                full_string[row['text']]=data
              

In [ ]:
full_string

In [ ]:
dict_new={}
for key,value in full_string.items():
    if value==value:
        print(key,value)
    